# Extract_Data

Il faut que organiser vos datas en DATA/DATA_0 DATA/DATA_1
Il faut changer la deuxième case en XBT ou ETH selon ce que vous voulez

In [5]:
coin = 'XBT_EUR.parquet'
I = 1

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_btc = pl.read_csv(f'../data/raw/DATA_{I}/{coin}_EUR.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/DATA_1/XBT_EUR.parquet_EUR.csv'

In [ ]:
def preprocessing_vectorized(df):
    n = len(df)
    block_size = 20

    # Créer un ID de bloc : 0,0,...,1,1,...,2,2,...
    row_id = np.repeat(np.arange((n + block_size - 1) // block_size), block_size)[:n]
    df = df.copy()
    df['row_id'] = row_id

    # Facultatif : aligner timestamp par bloc
    max_timestamps = df.groupby('row_id')['timestamp'].max().reset_index()
    max_timestamps.columns = ['row_id', 'max_timestamp']
    df = df.merge(max_timestamps, on='row_id')
    df['timestamp'] = df['max_timestamp']
    df = df.drop('max_timestamp', axis=1)

    return df


In [ ]:
df = preprocessing_vectorized(df_btc)

In [ ]:
# cf les doublons d'avant
dups = df.duplicated(subset=["timestamp", "row_id", "level", "side"])
print(df[dups])


            price    volume     timestamp side  level   row_id
5417008   2345.90  0.080451  1.748355e+09  bid      2   270850
5417009   2345.91  0.013785  1.748355e+09  ask      1   270850
5417010   2345.95  1.841000  1.748355e+09  ask      1   270850
5417011   2346.10  0.087000  1.748355e+09  bid      2   270850
5417027   2345.65  0.013788  1.748355e+09  bid      3   270851
...           ...       ...           ...  ...    ...      ...
93366669  2325.01  0.008065  1.748429e+09  ask      1  4668333
93366670  2325.02  0.190000  1.748429e+09  ask      1  4668333
93366679  2326.46  0.743225  1.748429e+09  bid     10  4668333
93366689  2325.01  0.008065  1.748429e+09  ask      1  4668334
93366690  2325.02  0.190000  1.748429e+09  ask      1  4668334

[18170 rows x 6 columns]


In [ ]:
df_clean = df[~dups]

In [ ]:
df_wide = df_clean.pivot_table(values=["price", "volume"], index=["timestamp", "row_id"], columns=["level", "side"])
# Flatten the multi-level column index
df_wide.columns = ['_'.join(map(str, col)).strip() for col in df_wide.columns.values]
df_wide = df_wide.reset_index()


MemoryError: Unable to allocate 715. MiB for an array with shape (93746050,) and data type int64

In [ ]:
df_wide

price                                               \
level                      10       9        8        7        6        5    
side                      bid      bid      bid      bid      bid      bid   
timestamp    row_id                                                          
1.748488e+09 0        2452.25  2452.44  2452.49  2452.50  2452.52  2452.54   
             1        2452.25  2452.44  2452.49  2452.50  2452.52  2452.54   
             2        2452.25  2452.44  2452.49  2452.50  2452.52  2452.54   
             3        2452.25  2452.44  2452.49  2452.50  2452.52  2452.54   
             4        2452.44  2452.49  2452.50  2452.52  2452.54  2452.61   
...                       ...      ...      ...      ...      ...      ...   
1.748523e+09 2684997  2375.26  2375.32  2375.39  2375.41  2375.47  2375.56   
             2684998  2375.25  2375.26  2375.32  2375.39  2375.41  2375.47   
             2684999  2375.25  2375.26  2375.32  2375.39  2375.41  2375.47   
             2685000  2375.23  2375.25  2375.26  2375.32  2375.39  2375.41   
             2685001  2375.23  2375.25  2375.26  2375.32  2375.39  2375.41   

                                                          ...    volume  \
level                      4        3        2        1   ...        1    
side                      bid      bid      bid      bid  ...       ask   
timestamp    row_id                                       ...             
1.748488e+09 0        2452.61  2452.77  2452.94  2453.02  ...  0.004709   
             1        2452.61  2452.77  2452.94  2453.02  ...  0.004709   
             2        2452.61  2452.77  2452.94  2453.02  ...  0.004709   
             3        2452.61  2452.77  2452.94  2453.02  ...  0.004709   
             4        2452.77  2452.94  2452.99  2453.02  ...  0.004709   
...                       ...      ...      ...      ...  ...       ...   
1.748523e+09 2684997  2375.66  2375.68  2375.88  2375.89  ...  0.019769   
             2684998  2375.66  2375.68  2375.88  2375.89  ...  0.019769   
             2684999  2375.66  2375.68  2375.88  2375.89  ...  0.019769   
             2685000  2375.66  2375.68  2375.88  2375.89  ...  0.019769   
             2685001  2375.66  2375.68  2375.88  2375.89  ...  0.019769   

                                                                           \
level                       2          3         4          5          6    
side                       ask        ask       ask        ask        ask   
timestamp    row_id                                                         
1.748488e+09 0        1.004171  36.244590  36.24223   1.924000  36.240142   
             1        1.004171  36.242230   1.92400  36.240142   0.434709   
             2        1.004171   3.400000  36.24223   1.924000  36.240142   
             3        1.004171  36.244319   3.40000  36.242230   1.924000   
             4        1.004171  36.244319   3.40000  36.242230   1.924000   
...                        ...        ...       ...        ...        ...   
1.748523e+09 2684997  0.607047   0.067985  19.60200  23.167000   0.000653   
             2684998  0.607047   0.067985  19.60200  23.167000   0.000653   
             2684999  0.607047   0.067985  19.60200  23.167000   0.000653   
             2685000  0.607047   0.067985  19.60200  23.167000   0.000653   
             2685001  0.607047  19.602000  23.16700   0.000653  26.737000   

                                                                
level                        7         8         9          10  
side                        ask       ask       ask        ask  
timestamp    row_id                                             
1.748488e+09 0         0.434709  0.621428  1.100373   9.936000  
             1         0.621428  1.100373  9.936000  19.140000  
             2         0.434709  0.621428  1.100373   9.936000  
             3        36.240142  0.434709  0.621428   1.100373  
             4        36.240142  0.434709  0.621428

In [ ]:
# Aplatir le multi-index en format souhaité
column_mapping = {}
for col in df_wide.columns:
    if "_" in col and col not in ["timestamp", "row_id"]:
        parts = col.split("_")
        if len(parts) >= 3:
            var = parts[0]
            level = parts[1]
            side = parts[2]
            new_name = f"level-{level}-{side}-{var}"
            column_mapping[col] = new_name

df_wide = df_wide.rename(columns=column_mapping)

In [ ]:
df_wide

level-10-bid-price  level-9-bid-price  \
timestamp    row_id                                           
1.748488e+09 0                   2452.25            2452.44   
             1                   2452.25            2452.44   
             2                   2452.25            2452.44   
             3                   2452.25            2452.44   
             4                   2452.44            2452.49   
...                                  ...                ...   
1.748523e+09 2684997             2375.26            2375.32   
             2684998             2375.25            2375.26   
             2684999             2375.25            2375.26   
             2685000             2375.23            2375.25   
             2685001             2375.23            2375.25   

                      level-8-bid-price  level-7-bid-price  level-6-bid-price  \
timestamp    row_id                                                             
1.748488e+09 0                  2452.49            2452.50            2452.52   
             1                  2452.49            2452.50            2452.52   
             2                  2452.49            2452.50            2452.52   
             3                  2452.49            2452.50            2452.52   
             4                  2452.50            2452.52            2452.54   
...                                 ...                ...                ...   
1.748523e+09 2684997            2375.39            2375.41            2375.47   
             2684998            2375.32            2375.39            2375.41   
             2684999            2375.32            2375.39            2375.41   
             2685000            2375.26            2375.32            2375.39   
             2685001            2375.26            2375.32            2375.39   

                      level-5-bid-price  level-4-bid-price  level-3-bid-price  \
timestamp    row_id                                                             
1.748488e+09 0                  2452.54            2452.61            2452.77   
             1                  2452.54            2452.61            2452.77   
             2                  2452.54            2452.61            2452.77   
             3                  2452.54            2452.61            2452.77   
             4                  2452.61            2452.77            2452.94   
...                                 ...                ...                ...   
1.748523e+09 2684997            2375.56            2375.66            2375.68   
             2684998            2375.47            2375.66            2375.68   
             2684999            2375.47            2375.66            2375.68   
             2685000            2375.41            2375.66            2375.68   
             2685001            2375.41            2375.66            2375.68   

                      level-2-bid-price  level-1-bid-price  ...  \
timestamp    row_id                                         ...   
1.748488e+09 0                  2452.94            2453.02  ...   
             1                  2452.94            2453.02  ...   
             2                  2452.94            2453.02  ...   
             3                  2452.94            2453.02  ...   
             4                  2452.99            2453.02  ...   
...                                 ...                ...  ...   
1.748523e+09 2684997            2375.88            2375.89  ...   
             2684998            2375.88            2375.89  ...   
             2684999            2375.88            2375.89  ...   
             2685000            2375.88            2375.89  ...   
             2685001            2375.88            2375.89  ...   

                      level-1-ask-volume  level-2-ask-volume  \
timestamp    row_id                                            
1.748488e+09 0                  0.004709            1.004171   
             1                

In [ ]:
'''

# Un plot qui vous permet de voir la distribution des timestamps

timestamps = df_wide["timestamp"].values
import pandas as pd
counts = pd.Series(timestamps).value_counts().sort_index()

# Échantillon aléatoire de 50 timestamps
counts_sample = counts.sample(50, random_state=42).sort_index()

# Plot
plt.figure(figsize=(12, 5))
counts_sample.plot(kind='bar')

plt.title("Nombre de row_id (échantillon) par timestamp")
plt.xlabel("Timestamp (sample)")
plt.ylabel("Nombre de row_id (snapshots)")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

'''

'\n\n# Un plot qui vous permet de voir la distribution des timestamps\n\ntimestamps = df_wide.index.get_level_values(\'timestamp\')\ncounts = timestamps.value_counts().sort_index()\n\n# Échantillon aléatoire de 50 timestamps\ncounts_sample = counts.sample(50, random_state=42).sort_index()\n\n# Plot\nplt.figure(figsize=(12, 5))\ncounts_sample.plot(kind=\'bar\')\n\nplt.title("Nombre de row_id (échantillon) par timestamp")\nplt.xlabel("Timestamp (sample)")\nplt.ylabel("Nombre de row_id (snapshots)")\nplt.grid(True, linestyle=\'--\', alpha=0.5)\nplt.tight_layout()\nplt.show()\n\n'

In [ ]:
df_wide.to_parquet(f"../data/preprocessed/DATA_{I}/{coin}_EUR.parquet", index=False)